# Nutrition

In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

## Raw Data

In [167]:
DATA_PATH_AVG = '../data/input/nutrients_avg.xlsx'

def process_data(data_path):
    df = pd.read_excel(data_path)
    time_column = df['Time']
    df_no_time = df.drop(columns='Time')
    X = df_no_time.to_numpy()
    sorted_data = np.sort(X, axis=0)[::-1]
    maxes = np.max(sorted_data, axis=0)
    normalized_data = sorted_data / maxes
    data_df = pd.DataFrame(normalized_data, columns=df_no_time.columns)
    data_df['Time'] = time_column.sort_values(ascending=True).reset_index(drop=True)
    return data_df, maxes

df_data_normalized, maxes = process_data(DATA_PATH_AVG)

df_data_normalized.head()

,B,Ca,Cu,Fe,K,Mg,Mn,Na,P,Zn,N,Time
0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0
1,0.975676,0.967033,0.757225,0.926220,0.984881,0.997118,0.963664,0.982721,0.964880,0.885609,0.947009,7
2,0.965541,0.958242,0.739884,0.925085,0.876890,0.981268,0.918244,0.906407,0.892791,0.833948,0.902171,14
3,0.864865,0.901099,0.624277,0.872872,0.280562,0.881844,0.438304,0.410367,0.510166,0.321033,0.631666,21


In [168]:
FERTILIZER_PATH = '../data/input/fertilizer.xlsx'

df_fertilizer = pd.read_excel(FERTILIZER_PATH)
df_fertilizer_normalized = df_fertilizer/maxes
df_fertilizer_normalized.head()

,B,Ca,Cu,Fe,K,Mg,Mn,Na,P,Zn,N
0,0.000210,0.000094,0.264246,0.001538,0.000125,0.000292,0.036727,0.000096,0.000286,0.021631,0.000050
1,0.032856,0.000095,0.181090,0.005791,0.000107,0.000319,0.010258,0.000165,0.000171,0.030520,0.000052
2,0.039601,0.000026,0.245943,0.003782,0.000117,0.000283,0.008109,0.000259,0.000115,0.180471,0.000008


## Data Preparation

In [184]:
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline, PchipInterpolator, Akima1DInterpolator

def interpolate_data(data_df, num_points=500, kind='cubic'):
    data = data_df.to_numpy()
    x_original = np.arange(data.shape[0])  # Original index positions
    x_interp = np.linspace(0, data.shape[0] - 1, num_points)  # Interpolation index positions
    
    # Initialize an array to store interpolated data
    Y_interp = np.zeros((num_points, data.shape[1]))
    
    # Interpolate each column
    if kind!='pchip':
        for col in range(data.shape[1]):
            f = interp1d(x_original, data[:, col], kind=kind)  # Use the specified interpolation method
            Y_interp[:, col] = f(x_interp)
    else:
        for col in range(data.shape[1]):
            pchip_interpolator = PchipInterpolator(x_original, data[:, col])  # Create PCHIP interpolator
            Y_interp[:, col] = pchip_interpolator(x_interp)


    # Create a DataFrame for the interpolated data
    interpolated_df = pd.DataFrame(Y_interp, columns=data_df.columns)

    return interpolated_df


df_data_normalized_interp = interpolate_data(df_data_normalized, num_points=21, kind='pchip')
 

In [183]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_curves_interactive(data_df):
    # Extract 'Time' column and other data
    time = data_df['Time']
    y = data_df.drop(columns='Time')  # Drop 'Time' to get only the data for plotting

    num_cols = y.shape[1]  # Number of columns to plot
    num_rows = (num_cols + 2) // 3  # Calculate number of rows needed for subplots

    # Create subplots figure with plotly
    fig = make_subplots(rows=num_rows, cols=3, subplot_titles=[col for col in y.columns],
                        vertical_spacing=0.15, horizontal_spacing=0.1)

    # Plot each column against 'Time'
    for i, col in enumerate(y.columns):
        row = i // 3 + 1  # Determine row position
        col_position = i % 3 + 1  # Determine column position

        fig.add_trace(
            go.Scatter(
                x=time,
                y=y[col],
                mode='lines',
                name=col,
                line=dict(width=2)
            ),
            row=row,
            col=col_position
        )

        # Customize the axis titles and range for each subplot
        fig.update_xaxes(title_text='Days', row=row, col=col_position)
        fig.update_yaxes(title_text='Concentration', row=row, col=col_position, range=[0, None])

    # Update layout for better readability
    fig.update_layout(
        height=300 * num_rows, width=900, title_text="Interactive Subplots of Concentrations Over Time",
        showlegend=False,  # Hide the legend since each subplot has a title
        template='plotly_white'
    )
    
    fig.show()

# Example usage:
df_data_interp = df_data_normalized_interp.drop(columns='Time')*maxes
df_data_interp['Time'] = df_data_normalized_interp['Time']
plot_curves_interactive(df_data_interp)


In [171]:
# def absolute_plant_uptake_from_rate(x_interp, plant_uptake_rate, start, end):
#     start_index = np.argmin(np.abs(x_interp - start)) # find the best index
#     end_index = np.argmin(np.abs(x_interp - end))
#     return np.sum(plant_uptake_rate[start_index:end_index,:],axis=0)/(end_index-start_index)

# absolute_uptake = absolute_plant_uptake_from_rate(x_interp, plant_uptake_rate, 0,1)
# print(f"{absolute_uptake = }")

In [172]:
def absolute_plant_uptake_during_interval(data_df, start, end):
    # Extract the 'Time' column
    time = data_df['Time'].to_numpy()
    
    # Find the nearest indices for the start and end times
    start_index = np.argmin(np.abs(time - start))
    end_index = np.argmin(np.abs(time - end))
    
    # Calculate uptake as the negative of the nutrient change in solution
    uptake = -(data_df.iloc[end_index, 0:-1] - data_df.iloc[start_index, 0:-1])  # Skip 'Time' column

    return uptake

uptake = absolute_plant_uptake_during_interval(df_data_normalized_interp, 0,21)

uptake.to_numpy()

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])

## Optimize

Consider a single time intervall $I = [t_0, T]$.

Unsing the model of the plant uptake we have obtained the the target nutrient concentration should be idealy $\vec T$.

Let us optimize the minimal total amount of fertilizer to reach the target under the constraint that the plant does not experience any deficite of nutrients.

$L = \min(|\vec{T} - (u_1 \vec{F}_1 + u_2 \vec{F}_2 + u_3 \vec{F}_3|)$

Constrain: $\vec{T} - (u_1 \vec{F}_1 + u_2 \vec{F}_2 + u_3 \vec{F}_3) \ge 0$

**Comments:**

1. Since the concentration have different scales, it might be reasonable to scale (transfrom) them using sklearn MaxMinScaler or StandartScaler.
Afterwards don't forget to apply the inverse transform to get correct values!

In [173]:
def nutrients(param, fertilizer, target):
    parameter = np.array(param).reshape(fertilizer.shape[0],1)
    weighted_fertilizers = np.multiply(fertilizer,parameter)
    total_fertilizers = weighted_fertilizers.sum(axis=0)
    value = target - total_fertilizers
    return value

def objective_function(param, fertilizer, target):
    parameter = np.array(param).reshape(fertilizer.shape[0],1)
    # print(f"{parameter.shape=}")
    # print(f"{fertilizer.shape=}")
    weighted_fertilizers = np.multiply(fertilizer,parameter)
    # print(f"{weighted_fertilizers.shape}")
    total_fertilizers = weighted_fertilizers.sum(axis=0)
    # print(f"{total_fertilizers.shape}")
    value = target - total_fertilizers
    return np.sum(np.abs(value))


def constraint_function(param, fertilizer, target):
    # Reshape the parameter vector and compute weighted fertilizers
    parameter = np.array(param).reshape(fertilizer.shape[0], 1)
    weighted_fertilizers = np.multiply(fertilizer, parameter)
    # Sum the weighted fertilizers to get the total contribution for each nutrient
    total_fertilizers = weighted_fertilizers.sum(axis=0)
    # Calculate the difference between the target and total fertilizers for each nutrient
    value = target - total_fertilizers
    # Return the difference (this will ensure SLSQP enforces the constraint for each nutrient)
    return value


def optimize(fertilizer, target, constraint_function):
    # Initial guess for the parameters, one for each row (fertilizer) in the fertilizer DataFrame
    initial_guess = np.zeros(fertilizer.shape[0])
    
    # Define bounds for each parameter (non-negative values)
    bounds = [(0, None) for _ in range(fertilizer.shape[0])]
    
    # Define the constraint dictionary using the provided constraint function
    constraints = {'type': 'ineq', 'fun': constraint_function, 'args': (fertilizer, target)}
    
    # Call the optimizer
    result = minimize(
        objective_function,
        initial_guess,
        args=(fertilizer, target),
        bounds=bounds,
        constraints=constraints,
        method='SLSQP'
    )
    return result  # Return optimized parameters and minimized objective value

In [174]:

uptake = absolute_plant_uptake_during_interval(df_data_normalized_interp, 1,2)

result = optimize(
    df_fertilizer_normalized.to_numpy(),
    uptake.to_numpy(),
    constraint_function
)

print("---------------------------------------------------------------------")
if result.success:
    print("Optimization was successful!")
else:
    print("Optimization failed. Reason:", result.message)

nutrient_values = nutrients(result.x, df_fertilizer_normalized.to_numpy(), uptake.to_numpy())
print("Scaled constraint values:", nutrient_values)
print("constraint values:", nutrient_values*maxes)


---------------------------------------------------------------------
Optimization was successful!
Scaled constraint values: [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
constraint values: [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]


## Iterative process

In [185]:
def calculate_nutrient_schedule(path_data, path_fertilizer, time_intervall_days):

    # Load data
    df_data_normalized, maxes = process_data(path_data)
    df_fertilizer_normalized = pd.read_excel(path_fertilizer)/maxes

    # Interpolate data to a timestep of days if necessary
    num_days = df_data_normalized['Time'].max()
    df_data_normalized_interp = interpolate_data(df_data_normalized, num_points=num_days, kind='pchip')
    
    # Number of fertilizing days
    num_fertilization_events = num_days // time_intervall_days

    results = dict()
    left_over = 0
    for i in range(num_fertilization_events):
        # uptake of the plant during the time intervall
        start = i*time_intervall_days
        end = (i+1)*time_intervall_days
        uptake = absolute_plant_uptake_during_interval(df_data_normalized_interp, start, end)
        target_concentration = np.abs(uptake.to_numpy() - left_over) # substract left over nutrients
        # optimize fertilizer amounts
        result = optimize(df_fertilizer_normalized,
                          target_concentration,
                          constraint_function)
        # calculate left over
        result.left_over = nutrients(result.x, df_fertilizer_normalized, target_concentration)
        # store result
        results[f"{int(end)}"] = result
        
    return results

optimization_results = calculate_nutrient_schedule(
                            path_data='../data/input/nutrients_avg.xlsx',
                            path_fertilizer='../data/input/fertilizer.xlsx',
                            time_intervall_days=7)

In [186]:
import plotly.graph_objects as go

def plot_element_stacked_interactive_streamlit(df):
    # Get element names (these are your columns)
    element_columns = df.columns

    # Prepare the x-axis labels (row index keys from the DataFrame)
    index_labels = df.index

    # Initialize the figure
    fig = go.Figure()

    # Initialize the bottom of the stack (start with zeros)
    bottom = np.zeros(len(index_labels))

    # Add each element as a trace
    for col in element_columns:
        element_values = df[col].values
        rounded_values = np.round(element_values, 3)  # Round to 3 decimal places

        # Add a bar trace for each element
        fig.add_trace(go.Bar(
            x=index_labels,
            y=element_values,
            name=col,
            text=[f'{val:.3f}' for val in rounded_values],  # Format values to 3 decimal places
            hoverinfo='text+name',  # Show value and label name on hover
            offsetgroup=0,
            base=bottom
        ))

        # Update the bottom position for the next stack
        bottom += element_values

    # Update layout for better readability and display all x-axis labels
    fig.update_layout(
        title='Element Excess',
        xaxis=dict(
            title='Day',
            type='category',                  # Use categorical axis for clear labeling
        ),
        yaxis=dict(title='Element Excess'),
        barmode='stack',
        legend=dict(title='Elements'),
        template='plotly_white',
        hovermode="x"
    )
    return fig


## Analysis

In [187]:
import pandas as pd

def create_left_over_df(optimization_results, maxes):
    data = {}
    for k, result in optimization_results.items():
        data[k] = result.left_over*maxes
    df = pd.DataFrame(data).T  # Transpose to have keys as row indexes
    return df

df_left_over = create_left_over_df(optimization_results, maxes=maxes)
fig = plot_element_stacked_interactive_streamlit(df_left_over)
fig.show()


In [190]:
def create_fertilizer_df(optimization_results):
    data = {}
    for k, result in optimization_results.items():
        data[k] = result.x
    df = pd.DataFrame(data).T  # Transpose to have keys as row indexes
    return df

df_fertilizer_usage = create_fertilizer_df(optimization_results)


fig = plot_element_stacked_interactive_streamlit(df_fertilizer_usage)
fig.show()

## Plot results

The next step is to calculate a schedule how to fertilize the plants.
As input we give the time intervall that we want to attent to the plants.
As a ouput we want the amount of each fertilizer we have to add for each time.

Comments:
- additional statistic would be nice
- some plots would be also nice

In [179]:
def calculate_fertilization_schedule(data_path, time_intervall_days, c_fertilizer1, c_fertilizer2, c_fertilizer3):
    
    # read and normalize data
    data, maxes = process_data(data_path=data_path)
    
    # normalize fartilizer
    norm_c_fertilizer1 = c_fertilizer1/maxes
    norm_c_fertilizer2 = c_fertilizer2/maxes
    norm_c_fertilizer3 = c_fertilizer3/maxes

    # interpolate the data
    x_interp, Y_interp = interpolate_data(data, num_points=4*7, kind='linear')
    # calculate the uptake of each nutrient of the plant
    # x_interp, plant_uptake_rate = calculate_nutrient_uptake_rate(x_interp, Y_interp, num_interp_points=500)
    # calculate 
    time_intervall_week = time_intervall_days/7
    num_fertilization_events = int(x_interp[-1] // time_intervall_week)

    residual = 0
    results = dict()

    # build schedule
    for i in range(num_fertilization_events):
        
        start = i*time_intervall_week
        end = (i+1)*time_intervall_week

        c_target_without_residuals = absolute_plant_uptake_during_interval(x_interp=x_interp, Y_interp=Y_interp, start=start ,end=end)

        # calculate the target considering residual (excess) nutrients in the solution
        c_target = np.abs(c_target_without_residuals - residual)

        # Define the constraint
        constraint = {
            'type': 'ineq',  # 'ineq' means the constraint function must return >= 0
            'fun': constraint_function,
            'args': (norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target)
        }

        # optimize 
        a_opt, b_opt, c_opt, minimized_value = optimize(norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target, constraint)
        
        params_opt = (a_opt, b_opt, c_opt)
        
        # calculate the excess nutrient after time period
        residual = nutrients(params_opt, norm_c_fertilizer1, norm_c_fertilizer2, norm_c_fertilizer3, c_target)

        # store the results
        results[f"{int(start*7)}"] = {'f1' : a_opt,
                                     'f2' : b_opt,
                                     'f3' : c_opt,
                                     'excess_1': residual[0],
                                     'excess_2': residual[1],
                                     'excess_3': residual[2],
                                     'excess_4': residual[3],
                                     'excess_5': residual[4],
                                     'excess_6': residual[5],
                                     'excess_7': residual[6],
                                     'excess_8': residual[7],
                                     'excess_9': residual[8],
                                     'excess_10': residual[9],
                                     'excess_11': residual[10],
                                    #  'excess' : (residual*maxes).round(3),
                                    #  'excess_normalized' : (residual).round(3),
                                     'objective_function': minimized_value}
    df = pd.DataFrame(results).T
    return df

c_fertilizer1 = np.abs(np.random.sample(11))*0.01
c_fertilizer2 = np.abs(np.random.sample(11))*0.01
c_fertilizer3 = np.abs(np.random.sample(11))*0.01
   

results = calculate_fertilization_schedule(data_path=DATA_PATH_AVG, time_intervall_days=1, c_fertilizer1=c_fertilizer1, c_fertilizer2=c_fertilizer2, c_fertilizer3=c_fertilizer3)

results.head()

ValueError: too many values to unpack (expected 2)

In [119]:
c_fertilizer1 = np.abs(np.random.sample(11*3).reshape(11,3))*0.01

# Reshape c_fertilizer1 to (1, 11) so that it matches the number of columns
c_fertilizer1 = c_fertilizer1.reshape(3, -1)

# Now create the DataFrame
df_fertilizer1 = pd.DataFrame(c_fertilizer1, columns=df.drop(columns='Time').columns)

df_fertilizer1.to_excel('../data/input/fertilizer.xlsx',index=None)

## Analyse results

In [129]:
def plot_excess_stacked(df_transposed):
    excess_labels = [f'Excess {i}' for i in range(len(df_transposed['excess'].iloc[0]))]
    days = df_transposed.index
    day_array = [int(day.split('_')[0])  for day in days]
    x = np.arange(len(days))  # the label locations for days
    width = 0.35  # The width of the bars

    # Plot stacked bar chart
    plt.figure(figsize=(10, 6))

    # Initialize the bottom of the stack
    bottom = np.zeros(len(days))

    # Plot each excess group on top of the previous one (stacked)
    for i, label in enumerate(excess_labels):
        excess_values = [df_transposed.loc[day, 'excess'][i] for day in df_transposed.index]
        plt.bar(x, excess_values, width=width, label=label, bottom=bottom)
        
        # Update the bottom to include the current excess values
        bottom += excess_values

    # Adding gridlines
    plt.grid(axis='y', which='both', linestyle='--', linewidth=0.7, color='grey')

    # Labeling
    plt.xlabel('Days')
    plt.ylabel('Excess Value')
    plt.title('Stacked Excess Values for Different Days')

    # Adjust x-ticks to display numeric labels (1, 2, 3, ...) instead of day_0, day_7, etc.
    # plt.xticks(x, labels=[i for i in range(len(days))])
    x_labels = [day+day_array[1] for day in day_array]
    plt.xticks(x, labels=x_labels)

    # Add legend and show the plot
    plt.legend(title='Excess Groups')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_excess_stacked(results)

In [ ]:
results['objective_function']